# ELGC-Net 快速训练 (GitHub集成版)

这个Notebook直接使用GitHub中的配置文件，无需手动创建任何文件。

## 步骤概览
1. 挂载Google Drive
2. 克隆项目
3. 安装依赖
4. 准备数据集
5. 开始训练
6. 评估模型（可选）
7. 保存结果（可选）

In [ ]:
# 1. 挂载Google Drive并设置环境
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# 检查GPU
!nvidia-smi

In [ ]:
# 2. 克隆项目（包含colab文件夹）
!rm -rf /content/elgcnet  # 清理旧文件
!git clone https://github.com/techmn/elgcnet.git /content/elgcnet
%cd /content/elgcnet

# 查看colab目录
!ls -la colab/

In [ ]:
# 3. 安装依赖
!pip install einops==0.3.2 timm==0.9.16 fvcore opencv-python scikit-learn tqdm -q

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

In [ ]:
# 4. 准备数据集（仅复制部分数据以加快训练）
!python colab/prepare_data.py --train_samples 100 --val_samples 25 --test_samples 25

In [ ]:
# 5. 开始训练
!python colab/train_colab.py

## 可选操作

In [ ]:
# 6. 评估模型（可选）
!python colab/eval_colab.py

In [ ]:
# 7. 保存结果到Drive（可选）
!python colab/save_results.py

In [ ]:
# 8. 监控GPU使用（可选）
# 在训练过程中，在新的cell中运行此代码
import sys
sys.path.append('/content/elgcnet/colab')
from monitor_training import monitor_gpu

# 监控60秒
monitor_gpu(duration=60, interval=2)

## 配置说明

### 修改训练参数
如需修改训练参数，编辑 `colab/colab_config.py`：

```python
class ColabConfig:
    BATCH_SIZE = 8      # 批次大小
    MAX_EPOCHS = 100    # 训练轮数
    LEARNING_RATE = 0.00031  # 学习率
```

### 数据集路径
确保数据集在Google Drive的以下位置：
```
/content/drive/My Drive/Change_Detection/LEVIR-CD-256/
```

### 故障排除
1. **GPU内存不足**：减小BATCH_SIZE
2. **找不到数据集**：检查Google Drive路径
3. **训练中断**：模型会自动保存，可以从检查点恢复